In [1]:
import scipy.stats as stats
import numpy as np
import math
from scipy.stats import norm
from scipy.stats import gamma

n=50

true_beta = np.transpose(stats.norm.rvs(loc=0,scale=1,size=2))
true_phi = stats.gamma.rvs(a=3,scale=1/2,size=1)

x = np.transpose(np.array([np.ones(n),stats.norm.rvs(loc=0,scale=1,size=n)]))
y = np.random.normal(x.dot(true_beta), np.sqrt(1/true_phi))


beta0 = stats.norm.rvs(loc=0,scale=1,size=2)
phi0 = stats.gamma.rvs(a=3,scale=1/2,size=1)
theta0 = np.hstack([beta0, phi0])

#Set hyperparameters
a = 3.0
b= 2.0


print(true_beta,true_phi)
print(theta0)

[-0.9552603   1.58480052] [ 1.16897416]
[ 0.11092014  0.01037721  1.73451572]


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
def log_joint(y, x, theta): 
    n = y.shape[0]
    return sum(norm.logpdf(y,loc=x.dot(theta[0:2]), scale=1/np.sqrt(theta[2])))+norm.logpdf(theta[0],loc=0,scale=1/np.sqrt(.01))+norm.logpdf(theta[1],loc=0,scale=1/np.sqrt(.01)) +gamma.logpdf(theta[2], a, scale=1/b) 

In [4]:
%load_ext cython

In [8]:
%%cython -a
import cython
import numpy as np
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import gamma


@cython.boundscheck(False)
@cython.wraparound(False)
def leapfrog(y,x,theta,double[:] r,double eps):
    cdef int a,n
    cdef double b
    a = 3
    b = 2.0
    n = y.shape[0]
    gradients = np.hstack([theta[2]*(np.transpose(y).dot(x)-np.transpose(x).dot(x).dot(theta[0:2]))-.01*theta[0:2],(.5*n+a-1)/theta[2]-b-0.5*np.transpose((y-x.dot(theta[0:2]))).dot(y-x.dot(theta[0:2]))])
    r_upd = r + 0.5*eps*gradients
    theta_upd = theta + eps * r_upd
    gradients = np.hstack([theta_upd[2]*(np.transpose(y).dot(x)-np.transpose(x).dot(x).dot(theta_upd[0:2]))-.01*theta_upd[0:2],(.5*n+a-1)/theta[2]-b-0.5*np.transpose((y-x.dot(theta_upd[0:2]))).dot(y-x.dot(theta_upd[0:2]))])
    r_upd += 0.5*eps*gradients
    return theta_upd, r_upd
def log_joint(y, x, theta): 
    cdef int n
    n = y.shape[0]
    return sum(norm.logpdf(y,loc=x.dot(theta[0:2]), scale=1/np.sqrt(theta[2])))+norm.logpdf(theta[0],loc=0,scale=1/np.sqrt(.01))+norm.logpdf(theta[1],loc=0,scale=1/np.sqrt(.01))+gamma.logpdf(theta[2],3, scale=0.5) 
def BuildTree(y,x,theta, r, u, v, j, eps):
    triangle_max = 1000 #recommend value pg 1359
    if(j==0):
        #base case, take one leapfrog step in direction v
        theta_prime,r_prime = leapfrog(y,x,theta,r,v*eps)
        if(u <= np.exp(log_joint(y,x,theta_prime)-(1/2)*r_prime.dot(r_prime))):
            n_prime = 1
        else:
            n_prime = 0
        if(log_joint(y,x,theta_prime)-(1/2)*r_prime.dot(r_prime) > u-triangle_max):
            s_prime = 1
        else:
            s_prime = 0
        return theta_prime,r_prime,theta_prime,r_prime,theta_prime,n_prime,s_prime    
    else:
        #recursion-build the left and right subtrees
        theta_minus,r_minus,theta_plus,r_plus,theta_prime,n_prime,s_prime = BuildTree(y,x,theta,r,u,v,j-1,eps)
        if(s_prime==1):
            if(v == -1):
                theta_minus,r_minus,dash1,dash2,theta_primep,n_primep,s_primep = BuildTree(y,x,theta_minus,r_minus,u,v,j-1,eps)
            else:
                dash1,dash2,theta_plus,r_plus,theta_primep,n_primep,s_primep = BuildTree(y,x,theta_plus,r_plus,u,v,j-1,eps)  
            p = np.exp(np.log(n_primep)-np.log(n_prime+n_primep))
            unif = np.random.uniform()
            if(p>u):
                theta_prime = theta_primep
            if((theta_plus-theta_minus).dot(r_minus) >= 0 and (theta_plus-theta_minus).dot(r_plus) >= 0):  
                s_prime = s_primep  
            else:
                s_prime = 0  
            n_prime = n_prime+n_primep
    return theta_minus,r_minus,theta_plus,r_plus,theta_prime,n_prime,s_prime

def NUTS_Eff(theta0, y,x, eps, int M):
    #cdef double theta_m[500][3]
    theta_m = np.zeros((M,3))
    theta_m[0,:] = theta0
    cdef int m,s,j,v_j
    cdef double u,p,unif,n,n_prime
    #cdef double r0[3]
    #cdef double r_minus[3]
    #cdef double r_plus[3]
    #cdef double theta_minus[3]
    #cdef double theta_plus[3]
    #cdef double dash1[3]
    #cdef double dash2[3]
    for m in range(1,M):
        r0 = stats.norm.rvs(size=3)
        u = np.random.uniform(low=0,high=np.exp(log_joint(y,x,theta_m[m-1,:])-(1/2)*r0.dot(r0)))
        theta_minus = theta_m[m-1,:]
        theta_plus = theta_m[m-1,:]
        theta_m[m,:] = theta_m[m-1,:]
        r_minus = r0
        r_plus = r0
        j=0
        n = 1
        s=1
        while(s==1):
            v_j = np.random.choice([-1,1])
            if(v_j==-1):
                theta_minus,r_minus,dash1,dash2,theta_prime,n_prime,s_prime = BuildTree(y,x,theta_minus,r_minus,u,v_j,j,eps)
            else:
                dash1,dash2,theta_plus,r_plus,theta_prime,n_prime,s_prime = BuildTree(y,x,theta_plus,r_plus,u,v_j,j,eps)
            if(s_prime == 1):
                p = min(1,n_prime/n)
                unif = np.random.uniform()
                if(p>u):
                    theta_m[m,:] = theta_prime
            n = n+n_prime
            if((theta_plus-theta_minus).dot(r_minus) >= 0 and (theta_plus-theta_minus).dot(r_plus) >= 0):
                s = s_prime
            else:
                s = 0
            j = j+1    
    return(theta_m)

In [9]:
M = 500
eps = .005

In [10]:
%%time
results = NUTS_Eff(theta0, y,x, eps, M)
print("truth", (true_beta, true_phi))
beta0_found = np.mean(results[round(4*M/5):(M-1),0])
beta1_found = np.mean(results[round(4*M/5):(M-1),1])
phi_found = np.mean(results[round(4*M/5):(M-1),2])
print("Mean after burn in we find",beta0_found,beta1_found ,phi_found)


truth (array([-0.9552603 ,  1.58480052]), array([ 1.16897416]))
Mean after burn in we find -0.944859045312 1.43843819416 1.0707875564
CPU times: user 47 s, sys: 65 ms, total: 47 s
Wall time: 47 s


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
betas_found = np.hstack([beta0_found,beta1_found])
y_sim = stats.norm.rvs(loc=x.dot(betas_found),scale=1/np.sqrt(phi_found))
#y_sim = stats.norm.rvs(loc=x*beta_found,scale=phi_found)

fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(x[:,1], y, s=10, c='b', marker="s", label='Observed')
ax1.scatter(x[:,1], y_sim, s=10, c='r', marker="o", label='Estimated')
plt.legend(loc='lower left');
plt.show()


In [ ]:
a = 3
b = 2

In [ ]:
print(gamma.logpdf(theta0[2], a, scale=1/b) )